# Chapter 14 - 반복형, 반복자, 제너레이터

## Sentence 버전 1. 단어 시퀀스

In [2]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    

In [2]:
s = Sentence('"The time has come, " the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [5]:
s[5]

'Walrus'

파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 iter(x)를 자동으로 호출한다.<br>
iter() 내장 함수는 다음 과정을 수행한다.<br>

1. 객체가 \_\_iter\_\_() 메서드를 구현하는지 확인하고, 이 메서드를 호출해서 반복자를 가져온다.
2. \_\_iter\_\_() 메서드가 구현되어 있지 않지만 \_\_getitem\_\_()이 구현되어 있다면, 파이썬은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자를 생성한다.
3. 이 과정이 모두 실패하면 TypeError가 발생한다.

## 반복형과 반복자

파이썬은 반복형 객체에서 반복자를 가져온다.<br>
반복자에 대한 표준 인터페이스는 다음과 같은 메서드 두 개를 정의한다.

- \_\_next\_\_(): 다음에 사용할 항목을 반환한다. 더 이상 항목이 남아있지 않으면 StopIteration을 발생시킨다.
- \_\_iter\_\_(): self를 반환한다. 그러면 for 루프 등 반복형이 필요한 곳에 반복자를 사용할 수 있게 해준다.

In [3]:
s2 = Sentence('Pig and Pepper')
it = iter(s2)
it

In [4]:
next(it)

'Pig'

In [5]:
next(it)

'and'

In [6]:
next(it)

'Pepper'

In [7]:
next(it)

StopIteration: 

In [8]:
list(it)

[]

In [9]:
list(iter(s2))

['Pig', 'and', 'Pepper']

## Sentence 버전 2. 고전적인 반복자

In [10]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

이 예제가 작동하는 데에는 SentenceIterator에서 \_\_iter\_\_()를 구현할 필요가 없지만, 올바른 구현이다.<br>
반복자는 \_\_next\_\_()와 \_\_iter\_\_() 메서드를 모두 구현해야 한다.

## Sentence 버전 3. 제너레이터 함수

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return


버전 2보다는 훨씬 짧지만 그리 느긋(Lazy)한 것은 아니다.<br>
좀 더 느긋하게 처리하도록 구현할 수 있다.

## Sentence 버전 4. 느긋한 구현

In [2]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()


## Sentence 버전 5. 제너레이터 표현식

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))


## 표준 라이브러리의 제너레이터 함수

|모듈|함수|설명|
|--|--|--|
|itertools|compress(it, selector_it)|두 개의 반복형을 병렬로 소비한다. selector_it의 해당 항목이 참된 값일 때마다 it에서 항목을 생성한다.|
|itertools|dropwhile(predicate, it)|predicate가 참된 값인 동안 항목들을 지나가면서 it를 소비한 후, 추가 검사 없이 남아 있는 항목을 모두 생성한다.|
|내장|filter(predicate, it)|predicate를 it의 각 항목에 적용해서 predicate(it)가 참된 값이면 각 항목을 생성한다. predicate가 None이면 참된 항목을 모두 생성한다.|
|itertools|filterfalse(predicate, it)|filter()와 같지만 반대 논리를 적용한다. predicate로 거짓된 값이 나오는 항목을 모두 생성한다.|
|itertools|islice(it.top)이나 islice(it, start, stop, step=1)|s[:stop]이나 s[start:stop:step]과 비슷하게, 반복할 수 있는 모든 객체에 느긋하게 연산을 적용해서 it의 슬라이스 항목을 생성한다.|
|itertools|takewhile(predicate, it)|predicate가 참된 값으로 계산되는 동안 모든 항목을 생성하고, 추가 검사 없이 멈춘다.|
|itertools|combinations(it, out_len)|it로 생성된 항목에서 out_len 개의 조합을 생성한다.|
|itertools|combinations_with_replacement(it, out_len)|반복된 항목들의 조합을 포함해서, it로 생성된 항목에서 out_len 개의 조합을 생성한다.|
|itertools|count(start=0, step=1)|start에서 시작해서 step만큼 증가시키며 숫자를 무한히 생성한다.|
|itertools|sycle(it)|각 항목의 사본을 저장한 후, 항목을 무한히 반복한다.|
|itertools|permutations(it, out_len=None)|it로 생성된 항목에서 out_len 개 항목의 조합을 생성한다. 기본적으로 out_len은 len(list(it))다.|
|itertools|repeat(item [, times])|times를 지정하면 times만큼, 아니면 주어진 item을 무한히 반복해서 생성한다.|
|itertools|groupby(it, key=None)|(<키>, <그룹>)의 튜플을 생성한다. 이때 <키>는 그룹화 기준, <그룹>은 그룹 안의 항목을 생성하는 제너레이터이다.|
|내장|reversed(seq)|seq 안의 항목을 뒤에서부터 역순으로 생성한다. seq는 시퀀스이거나 \_\_reversed\_\_() 특별 메서드를 구현해야 한다.|
|itertools|tee(it, n=2)|n개의 제너레이터로 구성된 튜플을 하나 생성한다. 각 제너레이터는 입력된 it를 독립적으로 생성한다.|